In [2]:
from ast import Lambda
import tensorflow as tf
import json

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [8]:
# 350, 300
wt1 = 350 
ht1 = 231

In [9]:
wo1 = 640
ho1 = 426

In [14]:
ar1 = wo1/ho1
ar1

1.5023474178403755

In [17]:
350/ar1

232.96875

In [7]:
wt2 = 276
ht2 = 300

In [6]:
wo2 = 586
ho2 = 640

In [13]:
ar2 = wo2/ho2
ar2

0.915625

In [19]:
300*ar2

274.6875

In [24]:
x = tf.random.uniform((426, 640, 3))
tf.shape(x)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([426, 640,   3])>

In [28]:
target_size = tf.constant([300, 350])
ar = tf.shape(x)[1]/tf.shape(x)[0]
if tf.argmax(tf.shape()[:-1]) == tf.argmax(target_size):
    y_scale = tf.constant([target_size])

TypeError: Missing required positional argument

In [29]:
[300, 350][tf.argmax([300, 350])]

350

In [23]:
tf.argmax([640, 586, 3])== tf.argmax([300, 350])

<tf.Tensor: shape=(), dtype=bool, numpy=False>